In [1]:
import requests, json, time
import pandas as pd
import signaturehelper
import warnings
warnings.filterwarnings('ignore')
from scrapy.http import TextResponse

#### 1. 키워드에 대한 검색량 데이터 수집

In [2]:
# 네이버 검색 api 사용
def get_header(method, uri, api_key, secret_key, customer_id):
    timestamp = str(round(time.time() * 1000))
    signature = signaturehelper.Signature.generate(timestamp, method, uri, SECRET_KEY)
    return {'Content-Type': 'application/json; charset=UTF-8', 'X-Timestamp': timestamp, 'X-API-KEY': API_KEY, 'X-Customer': str(CUSTOMER_ID), 'X-Signature': signature}

BASE_URL = 'https://api.naver.com'
API_KEY = '0100000000f36305849c456e4267326a0a16538fe962c8830e57d2b4701e915772c9b3047f'
SECRET_KEY = 'AQAAAADzYwWEnEVuQmcyagoWU4/pEk+igjD53k1nAH4rtUqO1g=='
CUSTOMER_ID = '2488414'

In [3]:
# keyword 입력시 keywordstool API 호출
def keyword_master(keyword):
    dic_return_kwd = {}
    uri = '/keywordstool'
    method = 'GET'
    prm = {'hintKeywords' : keyword , 'showDetail':1}

    # ManageCustomerLink Usage Sample
    r = requests.get(BASE_URL + uri, params=prm, headers=get_header(method, uri, API_KEY, SECRET_KEY, CUSTOMER_ID))
    r_data = r.json()
    print("response status_code = {}".format(r.status_code))
    return r_data

In [4]:
# 검색 키워드 생일선물
keyword = '생일선물'
df = keyword_master(keyword)
df = pd.DataFrame(df['keywordList'])

response status_code = 200


In [5]:
# 확인
df.head() 

,relKeyword,monthlyPcQcCnt,monthlyMobileQcCnt,monthlyAvePcClkCnt,monthlyAveMobileClkCnt,monthlyAvePcCtr,monthlyAveMobileCtr,plAvgDepth,compIdx
0,생일선물,8690,69800,11.1,15.7,0.14,0.03,15,높음
1,케이크토퍼,4650,29300,26.1,594.8,0.59,2.16,15,높음
2,차량용방향제,19900,112100,63.2,1273.5,0.36,1.21,15,높음
3,책상의자,8350,46100,34.5,434.5,0.47,1.06,15,높음
4,선물쇼핑백,330,2070,5.6,73.0,1.77,3.35,15,높음


#### 2.검색어에 대한 상품등록수 수집
- item_count : 등록된 상품수
- total_search_count : PC, Mobile 을 합친 검색량

In [6]:
# 정적페이지 > xpath로 등록된 상품수 크롤링

In [7]:
# 상품수 가져오기
item_count_list = []
def item_count(keyword):
    for word in keyword:
        # url keyword
        url =f'https://search.shopping.naver.com/search/all?frm=NVSHATC&origQuery=실내용자전거&pagingIndex=1&pagingSize=40&productSet=total&query={word}&sort=rel&timestamp=&viewType=list#'
        # req 변수에 저장 requests.get(url) 
        req = requests.get(url) 
        # TextResponse(url) > response(data)
        response = TextResponse(req.url, body=req.text, encoding='utf-8') 
        # xpath 불러오기 , list로 
        data = response.xpath('//*[@id="__next"]/div/div[2]/div/div[3]/div[1]/div[1]/ul/li[1]/a/span[1]/text()').extract()
        item_count_list.append(data)
        


In [8]:
# 20개 까지만 검색 키워드 상품수 가져오기 
keyword = df['relKeyword'].values
item_count(keyword[:20])


In [9]:
# keywords 리스트 형태로 담기
keywords = [i[0] for i in item_count_list]

In [10]:
# 검색어 20개 까지만 보기, 불필요한 컬럼 정리
columns =['relKeyword','monthlyPcQcCnt','monthlyMobileQcCnt']
result_df = df[columns]
result_df = result_df[:20]

In [11]:
# keywords 추가하고 ',' 제거
result_df['item_count'] = keywords
result_df['item_count'] = result_df['item_count'].apply(lambda data: data.replace(',',''))

In [12]:
# total_search_count : pc와 mobile 을 합친 검색량
result_df['total_search_count'] = result_df['monthlyPcQcCnt']  + result_df['monthlyMobileQcCnt']

In [13]:
# 20개의 검색키워드 데이터프레임 형태로 추출
result_df

,relKeyword,monthlyPcQcCnt,monthlyMobileQcCnt,item_count,total_search_count
0,생일선물,8690,69800,2885043,78490
1,케이크토퍼,4650,29300,244539,33950
2,차량용방향제,19900,112100,825339,132000
3,책상의자,8350,46100,885296,54450
4,선물쇼핑백,330,2070,511883,2400
5,보드게임,14500,95700,3278112,110200
6,여성가방쇼핑몰,530,5890,24783011,6420
7,한우선물세트,4300,41800,99217,46100
8,장난감,4880,57100,13505463,61980
9,화이트데이선물,5260,22900,82910,28160


#### 3. 경쟁률
- 아이템수/검색수 : 높을수록 경쟁이 심한 상품 키워드

In [14]:
# 경쟁률을 구하기 위해 item_count,total_search_count int형으로 변환
result_df['item_count'] = result_df['item_count'].apply(lambda data:data).astype(int)
result_df['total_search_count'] = result_df['total_search_count'].apply(lambda data:data).astype(int)

In [15]:
# competition_rate = item_count / total_search_count
result_df['competition_rate'] =  result_df['item_count'] / result_df['total_search_count']

In [16]:
# 경쟁률 순으로 오름차순 정렬 ,인덱스 초기화
result_df = result_df.sort_values(by='competition_rate', ascending=True, ignore_index=True)

In [17]:
# 생일선물을 검색한 결과
# 리마인드웨딩촬영, 가족사진, 토퍼제작 ,한우선물세트가 경쟁력이 낮다.
# 경쟁력과 더불어 사람들이 원하는 선물 종류도 함께 볼 수 있다.
result_df

,relKeyword,monthlyPcQcCnt,monthlyMobileQcCnt,item_count,total_search_count,competition_rate
0,리마인드웨딩촬영,940,13000,8143,13940,0.584146
1,가족사진,5190,44000,70344,49190,1.430047
2,토퍼제작,1480,11200,20709,12680,1.633202
3,한우선물세트,4300,41800,99217,46100,2.152213
4,화이트데이선물,5260,22900,82910,28160,2.944247
5,차량용방향제,19900,112100,825339,132000,6.252568
6,케이크토퍼,4650,29300,244539,33950,7.202916
7,생일토퍼,1990,11400,110452,13390,8.248842
8,어버이날,3980,22700,252300,26680,9.456522
9,차량방향제,5860,59300,825232,65160,12.664702


* 추가인사이트
    * 데이터 검색을 하면서 pc 검색 창이 복잡하다고 생각했고 보여지는 데이터 값으로도 pc 클릭수가 저조하다.
    * mobile이 편해서 많이 선호하기도 하면서 pc의 검색에 불편함이 있어서 그런것이 아닐까 생각한다.

In [18]:
# pc 와 mobile의 클릭수 평균을 비교했을 때 모바일이 더 많은 클릭수를 보이고 있다
pc_cnt_mean = result_df['monthlyPcQcCnt'].mean() 
mobile_cnt_mean = result_df['monthlyMobileQcCnt'].mean() 
print(f'pc_cnt_mean =  {pc_cnt_mean} , mobile_cnt_mean =  {mobile_cnt_mean}' )

pc_cnt_mean =  4846.0 , mobile_cnt_mean =  35252.0


In [19]:
# 모바일 클릭수 int형으로 변환
result_df['monthlyMobileQcCnt'] = result_df['monthlyMobileQcCnt'].apply(lambda data:data).astype(int)
# 모바일 클릭수 / 전체 클릭수 = 모바일 클릭률
result_df['mobile_search_ctr'] = result_df['monthlyMobileQcCnt'] / result_df['total_search_count']
# 한 눈에 보기 위해 df1 데이터 프레임 생성
df1 = result_df[['relKeyword','monthlyPcQcCnt','monthlyMobileQcCnt','mobile_search_ctr']]
df1

,relKeyword,monthlyPcQcCnt,monthlyMobileQcCnt,mobile_search_ctr
0,리마인드웨딩촬영,940,13000,0.932568
1,가족사진,5190,44000,0.894491
2,토퍼제작,1480,11200,0.883281
3,한우선물세트,4300,41800,0.906725
4,화이트데이선물,5260,22900,0.813210
5,차량용방향제,19900,112100,0.849242
6,케이크토퍼,4650,29300,0.863034
7,생일토퍼,1990,11400,0.851382
8,어버이날,3980,22700,0.850825
9,차량방향제,5860,59300,0.910068
